In [1]:
from functools import partial
from multiprocessing.dummy import Pool
from subprocess import call
import os.path as osp
from general_tools.in_out.basics import files_in_subdirs, create_dir
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from general_tools.plotting import read_transparent_png
import numpy as np

In [3]:
%matplotlib nbagg

In [8]:
# Parameters
binary = '/orions3-zfs/software/blender-2.74/blender '
binary_arg1 = '/orions4-zfs/projects/optas/Testing/Hao/blank.blend '

# Next arg is special for parts
binary_arg2 = '--background --python /orions4-zfs/projects/optas/Testing/Hao/render_adapted_for_part_annotations.py '

binary = binary + binary_arg1 + binary_arg2

top_out_image_dir = '/orions4-zfs/projects/optas/DATA/Images/Shape_Net_Core/2015_Summer_OUT/Rendered_Images/with_part_info'
create_dir(top_out_image_dir)
thread_num = 20

In [5]:
class_name = 'chair'
syn_id = snc_category_to_synth_id()[class_name]
top_in_dir = '/orions4-zfs/projects/optas/DATA/Meshes/Shape_Net_Core/chairs_with_parts/03001627/'
top_out_dir = osp.join(top_out_image_dir, syn_id)

03001627


In [6]:
search_pattern = '.obj$'
all_in_files = [f for f in files_in_subdirs(top_in_dir, search_pattern)]
print len(all_in_files)

3745


In [11]:
commands = []
for in_f in all_in_files:
    model_name = in_f.split('/')[-1].replace('.obj',  '')    
    out_dir = create_dir(osp.join(top_out_dir, model_name))    
    command = '%s %s %s' % (binary, in_f, out_dir)
    commands.append(command)    
print ('\nTotal number of commands is %d' % (len(commands)))


Total number of commands is 3745


In [18]:
pool = Pool(thread_num)
for idx, return_code in enumerate(pool.imap(partial(call, shell=True), commands)):
    if return_code != 0:
        print('command %d of %s failed' % (idx, commands[idx]))

In [62]:
def make_part_pixel_mask(raw_im):
    float_to_part = {66050: 'arms', 65795: 'legs', 66305: 'sit', 131585: 'back'}
    part_to_int = {'legs':1, 'seatt':2, 'back':3, 'arms':4}
    background = 131840
    
    im_temp = raw_im.astype(np.float)
    float_mask = im_temp[:,:,0] + (im_temp[:,:,1] + 1)**2 + (im_temp[:,:,2] + 2)**2
    uvals = np.unique(float_mask).astype(int)    
    dim_x, dim_y, _ = raw_im.shape
    part_mask = np.zeros(shape=(dim_x, dim_y), dtype=np.float32)        
    for i in uvals:
        if i == background:
            pass
        else:
            part_id = part_to_int[float_to_part[i]]
            part_mask[ np.where(float_mask==i) ] = part_id
    return part_mask

In [76]:
for in_f in all_in_files:
    model_name = in_f.split('/')[-1].replace('.obj',  '')    
    out_dir = osp.join(top_out_dir, model_name)
    in_f = osp.join(out_dir, view)
    raw_im = read_transparent_png(in_f)
    part_mask = make_part_pixel_mask(raw_im)
    np.save(osp.join(out_dir, 'mask'), part_mask)